In [ ]:
import pandas as pd
import numpy as np
import math



In [ ]:
#create clean, new dataframe
Dataframe = pd.DataFrame()
#open and create dataframe
Dataframe =pd.read_excel('CapstoneData.xlsx', sheet_name='Campaign Results')

#set index as customer number
Dataframe.set_index('Customer Number', inplace=True)

#drop negative period sales as irrelevent
Dataframe.drop(Dataframe[Dataframe['Campaign Period Sales']<0].index, inplace=True)

#Drop negative Historical Sales Volume
Dataframe.drop(Dataframe[Dataframe['Historical Sales Volume']<0].index, inplace=True)

#drop row with probably incorrect date
Dataframe['Date of First Purchase'].min()
Dataframe = Dataframe.drop(Dataframe[Dataframe['Date of First Purchase']=='1926-01-15 00:00:00'].index)
#print(Dataframe['Date of First Purchase'].min())

#Drop outlier number of prior year sales
Dataframe = Dataframe.drop(Dataframe[Dataframe['Number of Prior Year Transactions']==313].index)
#convert Y YY N to True False
names = ('Desk', 'Executive Chair', 'Standard Chair', 'Monitor', 'Printer', 'Computer', 'Insurance', 'Toner', 'Office Supplies')

for name in names:
    #print(name)
    #print(Dataframe[name].unique())

    Dataframe[name].fillna("N", inplace =True)
    #print(Dataframe[name].unique())
    Dataframe[name] = np.where(Dataframe[name].str.contains("Y"), True, False)

#convert categorical number of employees to integer
#print(Dataframe['Number of Employees'].unique())
#dictx = {'1-5':3,'6-10':8, '11-50':30, '51-100':75, '101-500':300, '500+':750, ' ':np.nan}
dictx = {'1-5':1,'6-10':2, '11-50':3, '51-100':4, '101-500':5, '500+':6, ' ':np.nan}
Dataframe['Number Employees'] = Dataframe['Number of Employees'].map(dictx) 
Dataframe = Dataframe.drop(columns='Number of Employees')
#fill in missing values with 8, because most common value (25 and 50 percentile)
Dataframe["Number Employees"].fillna(8, inplace = True) 


In [ ]:

#convert date of 1st purchase to quartile
from datetime import date
#print(Dataframe['Date of First Purchase'].dt.year.min())
Dataframe['ClientYear1stPurch'] = Dataframe['Date of First Purchase'].dt.year.max() - Dataframe['Date of First Purchase'].dt.year

Dataframe['Client Tenure'] =Dataframe['ClientYear1stPurch']*4//(Dataframe['ClientYear1stPurch'].max()+1)+1


In [ ]:
Dataframe = Dataframe.drop(columns = ['Date of First Purchase', 'ClientYear1stPurch'])

#drop irrelevent columns
Dataframe = Dataframe.drop(columns = 'Repurchase Method')
Dataframe = Dataframe.drop(columns = 'Last Transaction Channel')
#lanuage is 98% English or unknown.  Unknown is not reliable information.  thus we drop it.
Dataframe = Dataframe.drop(columns = 'Language')


In [ ]:

#create first y as Booleon sales
Dataframe['Sales_bool'] = np.where(Dataframe['Campaign Period Sales']>0, True, False)


Buyers = Dataframe.loc[Dataframe['Campaign Period Sales']>0]
Non_buyers = Dataframe.loc[Dataframe['Campaign Period Sales']==0]
Dataframe['Sales_num'] = Dataframe['Campaign Period Sales']
Dataframe = Dataframe.drop(columns = ['Campaign Period Sales'] )

In [ ]:

#now that they are extracted, drop columns
#Dataframe = Dataframe.drop(columns = ['Campaign Period Sales', 'Sales'])
#Buyers = Buyers.drop(columns = ['Campaign Period Sales','Sales'] )


In [ ]:
#Dataframe = Dataframe.drop(columns = 'Office Supplies')
"""
#drop other potentially useless columns - did not use this in final version


Dataframe = Dataframe.drop(columns=
['Do Not Direct Mail Solicit', 'Do Not Email', 'Do Not Telemarket',
       'Desk', 'Executive Chair',  'Insurance', 'Toner', 'Office Supplies'])
       """

In [ ]:
#Dataframe = Dataframe.drop(columns=['Computer', 'Printer'])
Dataframe.info()
Dataframe.describe()
print(Dataframe.iloc[0, 0:-2])
print(Dataframe.iloc[0, -2:-1])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#Reserve set for testing
Dataframe_train, Dataframe_reserve, y_df_train, y_reserve = train_test_split(Dataframe.iloc[:, 0:-2], Dataframe.iloc[:, -2:-1] , test_size=.25)


#train test split for Predicting Buyers
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,  y_test = train_test_split(Dataframe_train, y_df_train , test_size=.5)
#Make y_train_amt y_test_amt for LinearRegression
y_train_amt = y_train
y_train_amt = y_train_amt.merge(pd.DataFrame(Dataframe['Sales_num']), left_index=True, right_index=True)
y_train_amt.drop(columns = 'Sales_bool', inplace=True)

y_test_amt = y_test
y_test_amt = y_test_amt.merge(pd.DataFrame(Dataframe['Sales_num']), left_index=True, right_index=True)
y_test_amt.drop(columns = 'Sales_bool', inplace=True)

#scale data
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)


X_train_sub, X_validation_sub, y_train_sub, y_validation_sub = train_test_split(X_train_scale, y_train)

y_train_sub = np.ravel(y_train_sub)
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

gb = GradientBoostingClassifier(n_estimators=50, learning_rate = .1, max_features=6, max_depth = 5)
gb.fit(X_train_sub, y_train_sub)
print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_sub, y_train_sub)))
print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation_sub, y_validation_sub)))


predictions = gb.predict(X_test_scale)
cm = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(cm)

print("Classification Report")
print(classification_report(y_test, predictions))

from sklearn.model_selection import cross_val_score
feature_importances =  gb.feature_importances_
features = X_train.columns
feats= pd.DataFrame({'features':features, 'importance':feature_importances})
plt.figure(figsize =(8,2))
plt.bar(feats.features, feats.importance, color='lightblue')
plt.xticks(rotation = 90)
plt.show()

import matplotlib.pyplot as plt
labels = ['True Positives', 'False Positives']
sizes = [cm[1,1], cm[0,1]]
colors = ['gold', 'lightskyblue']
patches, texts = plt.pie(sizes, colors=colors, shadow=True, startangle=90)
plt.pie(sizes, colors=colors, shadow=True, startangle=90,autopct='%1.1f%%')
plt.legend(patches, labels, loc="best")

plt.axis('equal')
plt.tight_layout()
plt.show()


labels = ['True Negatives', 'False Negatives']
sizes = [cm[0,0], cm[1,0]]
colors = ['gold','lightskyblue']
patches, texts = plt.pie(sizes, colors=colors, shadow=True, startangle=90)
plt.pie(sizes, colors=colors, shadow=True, startangle=90,autopct='%1.1f%%')
plt.legend(patches, labels, loc="best")

plt.axis('equal')
plt.tight_layout()
plt.show()

Chance_of_Sale = cm[1,1]/(cm[1,1]+cm[0,1])
print(Chance_of_Sale)

In [ ]:
Real_Buyers_train = pd.DataFrame(X_train.iloc[:, 0:17])
Real_Buyers_train['IsBuyer'] = y_train
Real_Buyers_train.drop(Real_Buyers_train[Real_Buyers_train['IsBuyer']==False].index, inplace=True)
Real_Buyers_train = Real_Buyers_train.merge(pd.DataFrame(Dataframe['Sales_num']), left_index=True, right_index=True)
y_train_real = Real_Buyers_train['Sales_num']
Real_Buyers_train.drop(columns = 'IsBuyer', inplace=True)
Real_Buyers_train.drop(columns = 'Sales_num',inplace=True)


Expected_Buyers_test = pd.DataFrame(X_test.iloc[:, 0:17])
Expected_Buyers_test['Pred']=predictions
#Expected_Buyers_test.drop(Expected_Buyers_test[Expected_Buyers_test['Pred']==False].index,inplace=True)
Expected_Buyers_test = Expected_Buyers_test.merge(pd.DataFrame(Dataframe['Sales_num']), left_index=True, right_index=True)
y_test_exp= Expected_Buyers_test['Sales_num']
Expected_Buyers_test.drop(columns = 'Pred', inplace=True)
Expected_Buyers_test.drop(columns = 'Sales_num',inplace=True)
#Expected_Buyers_test.drop(columns = 'Sales_Amt',inplace=True)

Real_Buyers_train.info()
Expected_Buyers_test.info()

In [ ]:
"""
#train test split for LinReg (buyers only)
Buyers = Dataframe.loc[Dataframe['Sales_num']>0]
from sklearn.model_selection import train_test_split
X_train_buy, X_test_buy, y_train_buy,  y_test_buy = train_test_split(Buyers.iloc[:, 0:-2], Buyers.iloc[:, -1], test_size=.5)
#train_test split for all
X_train_all, X_test_all, y_train_all,  y_test_all = train_test_split(Dataframe.iloc[:, 0:-2], Dataframe.iloc[:, -1], test_size=.5)
"""


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, r2_score, median_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler= MinMaxScaler()
X_train_scale_buy = scaler.fit_transform(Real_Buyers_train)
#X_test_scale_buy = scaler.transform(Expected_Buyers_test)
X_test_scale_buy = scaler.transform(Expected_Buyers_test)
model_buy=LinearRegression().fit(X_train_scale_buy, y_train_real)

y_pred_buy = model_buy.predict(X_test_scale_buy)
#y_pred_non_buyers = model_buy.predict(X_test_scale_non_buy_pred)
print(r2_score(y_test_exp, y_pred_buy))

print(median_absolute_error(y_test_exp, y_pred_buy))


print(list(zip(X_train.columns, model_buy.coef_)))

from sklearn.model_selection import cross_val_score
feature_importances =  model_buy.coef_
features = Real_Buyers_train.columns
feats= pd.DataFrame({'features':features, 'importance':feature_importances})
plt.figure(figsize =(8,2))
plt.bar(feats.features, feats.importance, color='lightblue')
plt.xticks(rotation = 90)
plt.show()


In [ ]:



"""
start = 0
end = 1000

plt.scatter(y_pred_buy[start:end, ], y_test_buy[start:end, ],  color='gray')
#plt.plot(X_test, y_pred, color='red', linewidth=2)
plt.show()


plt.scatter(y_pred_all[start:end, ], y_test_all[start:end, ],  color='gray')
#plt.plot(X_test, y_pred, color='red', linewidth=2)
plt.show()

"""

In [ ]:
Lift= pd.DataFrame()
Lift = pd.DataFrame(X_test)
Lift['Pred']=predictions
Lift['Sales_Amt']= Dataframe['Sales_num'].loc[Lift.index & Dataframe.index]
Temp = pd.DataFrame(Expected_Buyers_test)
Temp['Pred_Amt'] = y_pred_buy

Lift['Pred_Amt'] = Temp['Pred_Amt'].loc[Lift.index & Temp.index]

Lift['Pred_Amt'].fillna(0, inplace=True)

Lift['Exp_Profit'] = Lift['Pred_Amt']*.22*Chance_of_Sale - np.where(Lift['Pred_Amt']>0, 8.4, 0)*Chance_of_Sale - 45.65
#Lift['Actual_Profit'] = Lift['Sales_Amt']*.22 - np.where(Lift['Sales_Amt']>0, 8.4, 0) - 45.65

Lift['Sales_bool'] = np.where(Lift['Sales_Amt']>0, True, False)
Lift['GrossProfit'] = Lift['Sales_Amt']*.22
Lift['Trans_cost'] = Lift['Sales_bool']*8.4
Lift['Camp_cost'] = 45.65
Lift['Actual_Profit']= Lift['GrossProfit']-Lift['Trans_cost']-Lift['Camp_cost']



Lift_Pred_Sale= Lift[(Lift['Pred']==True) & (Lift['Pred_Amt']>1)]
Lift_Pred_Sale['decile'] = pd.qcut(Lift_Pred_Sale['Pred_Amt']*-1, 10, labels=False,duplicates='drop')+1
Lift_Pred_No_Sale =Lift.loc[((Lift['Pred']==True) & (Lift['Pred_Amt']<1)) |  (Lift['Pred']==False)]  

#Lift_Pred_No_Sale.drop(Lift.index == isin(Lift_Pred_Sale.index), inplace = True)
Lift_Pred_No_Sale['decile'] = pd.qcut(Lift_Pred_No_Sale['Pred_Amt']*-1, 10, labels=False,duplicates='drop')+1

#@song0089 - Ya, so use df = df[df.index.isin(df1.index)]
#Lift_Pred_Sale.head(20)
#Lift.info()  Sales_Amt	Pred_Amt	Exp_Profit	Actual_Profit
#Lift_Pred_Sale.groupby("decile").min()
#Lift_Pred_No_Sale.groupby('decile').mean()

In [ ]:
Pred_Sale_means = Lift_Pred_Sale.groupby("decile").mean()
Pred_Sale_sum = Lift_Pred_Sale.groupby("decile").sum()
Pred_No_Sale_sum = Lift_Pred_No_Sale.groupby("decile").sum()
Pred_Sale_sum
#Pred_Sale_means.loc[Pred_Sale_means['Exp_Profit']>0]


In [ ]:
Lift_Pred_No_Sale.groupby("decile").min()

In [ ]:
clmnames= Lift.columns
for name in clmnames:
    fig=plt.figure()
    ax=fig.add_axes([0,0,1,1])
    ax.scatter(Lift[name], Lift['Pred_Amt'], color='r')
    #ax.scatter(grades_range, boys_grades, color='b')
    ax.set_xlabel(name)
    ax.set_ylabel('Sales')
    ax.set_title('scatter plot')
    plt.show()

In [ ]:
Pred_Sale_sum.to_excel('Pred_Sale_sum.xlsx')

Pred_No_Sale_sum.to_excel('Pred_No_Sale_sum.xlsx')
Lift.to_excel('Lift.xlsx')

In [ ]:
Lift_Pred_Sale[Lift_Pred_Sale['decile']<7]
Lift_Pred_Sale['Targeted']=np.where(Lift_Pred_Sale['decile']<7, True, False)
Lift_Pred_No_Sale['Targeted']=np.where(Lift_Pred_No_Sale['decile']<3, True, False )


In [ ]:
#Lift_Pred_Sale['Targeted']=np.where(Lift_Pred_Sale['decile']<7, True, False)
#Lift_Pred_No_Sale['Targeted']=np.where(Lift_Pred_No_Sale['decile']<3, True, False )

#frames = [Lift_Pred_No_Sale, Lift_Pred_Sale]
#All= pd.concat(frames)
All_sum = All.mean().transpose()
All_sum_Targ = All.groupby(['Targeted']).mean()
All_sum_Targ.to_excel('All_sum_Targ.xlsx')

In [ ]:
#now check on resrverd data
# Dataframe_reserve, y_reserve 
Dataframe_reserve_scale = scaler.transform(Dataframe_reserve.iloc[:, 0:16])

predictions_reserved = gb.predict(Dataframe_reserve_scale)
y_pred_res = model_buy.predict(Dataframe_reserve_scale)
Lift= pd.DataFrame()
Lift = pd.DataFrame(Dataframe_reserve)
Lift['Pred']=predictions_reserved
Lift['Sales_Amt']= Dataframe['Sales_num'].loc[Lift.index & Dataframe.index]

Lift['Pred_Amt'] = y_pred_res
Lift['Pred_Amt'].fillna(0, inplace=True)

Lift['Exp_Profit'] = Lift['Pred_Amt']*.22*Chance_of_Sale - np.where(Lift['Pred_Amt']>0, 8.4, 0)*Chance_of_Sale - 45.65
#Lift['Actual_Profit'] = Lift['Sales_Amt']*.22 - np.where(Lift['Sales_Amt']>0, 8.4, 0) - 45.65

Lift['Sales_bool'] = np.where(Lift['Sales_Amt']>0, True, False)
Lift['GrossProfit'] = Lift['Sales_Amt']*.22
Lift['Trans_cost'] = Lift['Sales_bool']*8.4
Lift['Camp_cost'] = 45.65
Lift['Actual_Profit']= Lift['GrossProfit']-Lift['Trans_cost']-Lift['Camp_cost']



Lift_Pred_Sale= Lift[(Lift['Pred']==True) & (Lift['Pred_Amt']>1)]
Lift_Pred_Sale['decile'] = pd.qcut(Lift_Pred_Sale['Pred_Amt']*-1, 10, labels=False,duplicates='drop')
Lift_Pred_No_Sale =Lift.loc[((Lift['Pred']==True) & (Lift['Pred_Amt']<1)) |  (Lift['Pred']==False)]  

#Lift_Pred_No_Sale.drop(Lift.index == isin(Lift_Pred_Sale.index), inplace = True)
Lift_Pred_No_Sale['decile'] = pd.qcut(Lift_Pred_No_Sale['Pred_Amt']*-1, 10, labels=False,duplicates='drop')+1

Pred_Sale_means = Lift_Pred_Sale.groupby("decile").mean()
Pred_Sale_sum = Lift_Pred_Sale.groupby("decile").sum()
Pred_No_Sale_sum = Lift_Pred_No_Sale.groupby("decile").sum()
Pred_No_Sale_sum
#Pred_Sale_means.loc[Pred_Sale_means['Exp_Profit']>0]
#Lift_Pred_No_Sale.groupby("decile").count()
#Lift.head()

In [ ]:
"""
#used to determine best parameters for GradientBoostingClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler

# train with Gradient Boosting algorithm
#train test split for Predicting Buyers
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,  y_test = train_test_split(Dataframe.iloc[:, 0:-2], Dataframe.iloc[:, -2:-1] , test_size=.5)

# compute the accuracy scores on train and validation sets when training with different learning rates
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

# split training feature and target sets into training and validation subsets
from sklearn.model_selection import train_test_split

X_train_sub, X_validation_sub, y_train_sub, y_validation_sub = train_test_split(X_train_scale, y_train, random_state=0)

y_train_sub = np.ravel(y_train_sub)
y_validation_sub=np.ravel(y_validation_sub)
learning_rates = [.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=50, learning_rate = learning_rate, max_features=5, max_depth = 5)
    gb.fit(X_train_sub, y_train_sub)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_sub, y_train_sub)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation_sub, y_validation_sub)))
    
    #feature_importances =  gb.feature_importances_
    #features = X_train.columns
    #feats= pd.DataFrame({'features':features, 'importance':feature_importances})
    #plt.figure(figsize =(8,2))
    #plt.bar(feats.features, feats.importance, color='lightblue')
    #plt.xticks(rotation = 90)
    #plt.show()
    
    print()
    
max_features = [3,4,5,6,7]
for max_f in max_features:
    gb = GradientBoostingClassifier(n_estimators=50, learning_rate = .1, max_features=max_f, max_depth = 5)
    gb.fit(X_train_sub, y_train_sub)
    print("max_features: ", max_f)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_sub, y_train_sub)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation_sub, y_validation_sub)))
    print()
  
max_depths = [3,4,5,6,7]
for max_d in max_depths:
    gb = GradientBoostingClassifier(n_estimators=50, learning_rate = .1, max_features=5, max_depth = max_d)
    gb.fit(X_train_sub, y_train_sub)
    print("max_depths: ", max_d)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_sub, y_train_sub)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation_sub, y_validation_sub)))
    print
    
n_est = [10,50,100,500,1000]
for n in n_est:
    gb = GradientBoostingClassifier(n_estimators=n, learning_rate = .1, max_features=5, max_depth = 5)
    gb.fit(X_train_sub, y_train_sub)
    print("n_est: ", n)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_sub, y_train_sub)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation_sub, y_validation_sub)))
    print()
    
"""